#Packages

In [72]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
! pip install contractions
import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [73]:
pip install --upgrade scipy numpy

In [74]:
pip install emoji --upgrade

In [75]:
import os
import re
import string
import emoji
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

#Dataset

In [76]:
train_data = pd.read_csv("/content/train.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
valid_data = pd.read_csv("/content/dev.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
test_data = pd.read_csv("/content/test.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])

In [77]:
train_data.head()

,Text,Class,ID
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj


In [78]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43410 entries, 0 to 43409
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    43410 non-null  object
 1   Class   43410 non-null  object
 2   ID      43410 non-null  object
dtypes: object(3)
memory usage: 1017.5+ KB


In [79]:
train_data['Class']

0        27
1        27
2         2
3        14
4         3
         ..
43405    18
43406     6
43407     3
43408    13
43409    17
Name: Class, Length: 43410, dtype: object

In [80]:
train_data['Class List'] = train_data['Class'].apply(lambda x: x.split(','))
train_data['Class Length'] = train_data['Class List'].apply(lambda x: len(x))

valid_data['Class List'] = valid_data['Class'].apply(lambda x: x.split(','))
valid_data['Class Length'] = valid_data['Class List'].apply(lambda x: len(x))

test_data['Class List'] = test_data['Class'].apply(lambda x: x.split(','))
test_data['Class Length'] = test_data['Class List'].apply(lambda x: len(x))

In [81]:
train_data.head()

,Text,Class,ID,Class List,Class Length
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1
3,To make her feel threatened,14,ed7ypvh,[14],1
4,Dirty Southern Wankers,3,ed0bdzj,[3],1


In [82]:
print(train_data['Class Length'].max())
print(valid_data['Class Length'].max())
print(test_data['Class Length'].max())

5
4
4


In [83]:
train_data[train_data['Class Length']==5]

,Text,Class,ID,Class List,Class Length
7873,Yeah I probably would've started crying on the...,"9,12,14,19,25",ee6lqiq,"[9, 12, 14, 19, 25]",5


In [84]:
train_data["Class Length"].value_counts()

1    36308
2     6541
3      532
4       28
5        1
Name: Class Length, dtype: int64

GoEmotion Taxonomy

In [85]:
emotion_file = open("/content/emotions.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [86]:
def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

In [87]:
train_data['Emotions'] = train_data['Class List'].apply(idx2class)
valid_data['Emotions'] = valid_data['Class List'].apply(idx2class)
test_data['Emotions'] = test_data['Class List'].apply(idx2class)

In [88]:
train_data.head()

,Text,Class,ID,Class List,Class Length,Emotions
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger]
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear]
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance]


In [89]:
train_data["Emotions"].value_counts()

[neutral]                              12823
[admiration]                            2710
[approval]                              1873
[gratitude]                             1857
[amusement]                             1652
                                       ...  
[disgust, excitement]                      1
[admiration, caring, optimism]             1
[disappointment, optimism, sadness]        1
[excitement, joy, pride]                   1
[desire, disgust, love]                    1
Name: Emotions, Length: 711, dtype: int64

In [90]:
train_data['GoEmotions'] = train_data['Emotions']
valid_data['GoEmotions'] = valid_data['Emotions']
test_data['GoEmotions'] = test_data['Emotions']

In [91]:
train_data.head()

,Text,Class,ID,Class List,Class Length,Emotions,GoEmotions
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger]
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear]
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[annoyance]


In [92]:
train_data["GoEmotions"].value_counts()

[neutral]                              12823
[admiration]                            2710
[approval]                              1873
[gratitude]                             1857
[amusement]                             1652
                                       ...  
[disgust, excitement]                      1
[admiration, caring, optimism]             1
[disappointment, optimism, sadness]        1
[excitement, joy, pride]                   1
[desire, disgust, love]                    1
Name: GoEmotions, Length: 711, dtype: int64

In [93]:
for i in emotion_list:
  train_data[i]=np.zeros((len(train_data),1))
  valid_data[i] = np.zeros((len(valid_data),1))
  test_data[i] = np.zeros((len(test_data),1))

In [94]:
for i in emotion_list:
    train_data[i] = train_data['GoEmotions'].apply(lambda x: 1 if i in x else 0)
    valid_data[i] = valid_data['GoEmotions'].apply(lambda x: 1 if i in x else 0)
    test_data[i] = test_data['GoEmotions'].apply(lambda x: 1 if i in x else 0)

In [95]:
train_data.head()

,Text,Class,ID,Class List,Class Length,Emotions,GoEmotions,admiration,amusement,anger,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[annoyance],0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Ekman Taxonomy

In [96]:
#opening the file
with open("/content/ekman_mapping.json") as file:
    ekman_mapping = json.load(file)
print(f"ekman mapping:{ekman_mapping}")

ekman mapping:{'anger': ['anger', 'annoyance', 'disapproval'], 'disgust': ['disgust'], 'fear': ['fear', 'nervousness'], 'joy': ['joy', 'amusement', 'approval', 'excitement', 'gratitude', 'love', 'optimism', 'relief', 'pride', 'admiration', 'desire', 'caring'], 'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'], 'surprise': ['surprise', 'realization', 'confusion', 'curiosity']}


In [97]:
def E_EmotionMapping(emotion_list):
    map_list = []

    for i in emotion_list:
        if i in ekman_mapping['anger']:
            map_list.append('e_anger')
        if i in ekman_mapping['disgust']:
            map_list.append('e_disgust')
        if i in ekman_mapping['fear']:
            map_list.append('e_fear')
        if i in ekman_mapping['joy']:
            map_list.append('e_joy')
        if i in ekman_mapping['sadness']:
            map_list.append('e_sadness')
        if i in ekman_mapping['surprise']:
            map_list.append('e_surprise')
        if i == 'neutral':
            map_list.append('e_neutral')

    return map_list

In [98]:
train_data['Ekman_Emotions'] = train_data['Emotions'].apply(E_EmotionMapping)
valid_data['Ekman_Emotions'] = valid_data['Emotions'].apply(E_EmotionMapping)
test_data['Ekman_Emotions'] = test_data['Emotions'].apply(E_EmotionMapping)

In [99]:
train_data['Ekman_Emotions'].value_counts()

[e_joy]                                     12920
[e_neutral]                                 12823
[e_anger]                                    3878
[e_surprise]                                 3553
[e_joy, e_joy]                               2129
                                            ...  
[e_joy, e_sadness, e_sadness, e_neutral]        1
[e_surprise, e_sadness, e_neutral]              1
[e_surprise, e_sadness, e_surprise]             1
[e_joy, e_anger, e_joy, e_surprise]             1
[e_joy, e_joy, e_sadness, e_joy]                1
Name: Ekman_Emotions, Length: 166, dtype: int64

In [100]:
train_data.head()

,Text,Class,ID,Class List,Class Length,Emotions,GoEmotions,admiration,amusement,anger,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,Ekman_Emotions
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,...,0,0,0,0,0,0,0,0,1,[e_neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,...,0,0,0,0,0,0,0,0,1,[e_neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],0,0,1,...,0,0,0,0,0,0,0,0,0,[e_anger]
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,0,...,0,0,0,0,0,0,0,0,0,[e_fear]
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[annoyance],0,0,0,...,0,0,0,0,0,0,0,0,0,[e_anger]


In [101]:
for i in ['e_anger', 'e_disgust', 'e_fear', 'e_joy', 'e_sadness', 'e_surprise','e_neutral']:
  train_data[i] = np.zeros((len(train_data),1))
  valid_data[i] = np.zeros((len(valid_data),1))
  test_data[i] = np.zeros((len(test_data),1))

In [102]:
for i in ['e_anger', 'e_disgust', 'e_fear', 'e_joy', 'e_sadness', 'e_surprise','e_neutral']:
    train_data[i] = train_data['Ekman_Emotions'].apply(lambda x: 1 if i in x else 0)
    valid_data[i] = valid_data['Ekman_Emotions'].apply(lambda x: 1 if i in x else 0)
    test_data[i] = test_data['Ekman_Emotions'].apply(lambda x: 1 if i in x else 0)

In [103]:
train_data.head()

,Text,Class,ID,Class List,Class Length,Emotions,GoEmotions,admiration,amusement,anger,...,surprise,neutral,Ekman_Emotions,e_anger,e_disgust,e_fear,e_joy,e_sadness,e_surprise,e_neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,...,0,1,[e_neutral],0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,...,0,1,[e_neutral],0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],0,0,1,...,0,0,[e_anger],1,0,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,0,...,0,0,[e_fear],0,0,1,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[annoyance],0,0,0,...,0,0,[e_anger],1,0,0,0,0,0,0


#Sentiments groups

In [104]:
with open("/content/sentiment_mapping.json") as file:
    sentiment_mapping = json.load(file)
sentiment_mapping

{'positive': ['amusement',
  'excitement',
  'joy',
  'love',
  'desire',
  'optimism',
  'caring',
  'pride',
  'admiration',
  'gratitude',
  'relief',
  'approval'],
 'negative': ['fear',
  'nervousness',
  'remorse',
  'embarrassment',
  'disappointment',
  'sadness',
  'grief',
  'disgust',
  'anger',
  'annoyance',
  'disapproval'],
 'ambiguous': ['realization', 'surprise', 'curiosity', 'confusion']}

In [105]:
def S_EmotionMapping(emotion_list):
    map_list = []

    for i in emotion_list:
        if i in sentiment_mapping['positive']:
            map_list.append('s_positive')
        if i in sentiment_mapping['negative']:
            map_list.append('s_negative')
        if i in sentiment_mapping['ambiguous']:
            map_list.append('s_ambiguous')
        if i == 'neutral':
            map_list.append('s_neutral')

    return map_list

In [106]:
train_data['Sentiment_Emotions'] = train_data['Emotions'].apply(S_EmotionMapping)
valid_data['Sentiment_Emotions'] = valid_data['Emotions'].apply(S_EmotionMapping)
test_data['Sentiment_Emotions'] = test_data['Emotions'].apply(S_EmotionMapping)

In [107]:
train_data.head()

,Text,Class,ID,Class List,Class Length,Emotions,GoEmotions,admiration,amusement,anger,...,neutral,Ekman_Emotions,e_anger,e_disgust,e_fear,e_joy,e_sadness,e_surprise,e_neutral,Sentiment_Emotions
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,...,1,[e_neutral],0,0,0,0,0,0,1,[s_neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,...,1,[e_neutral],0,0,0,0,0,0,1,[s_neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],0,0,1,...,0,[e_anger],1,0,0,0,0,0,0,[s_negative]
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,0,...,0,[e_fear],0,0,1,0,0,0,0,[s_negative]
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[annoyance],0,0,0,...,0,[e_anger],1,0,0,0,0,0,0,[s_negative]


In [108]:
train_data["Sentiment_Emotions"].value_counts()

[s_positive]                                                    12920
[s_neutral]                                                     12823
[s_negative]                                                     7012
[s_ambiguous]                                                    3553
[s_positive, s_positive]                                         2129
                                                                ...  
[s_ambiguous, s_ambiguous, s_negative, s_ambiguous]                 1
[s_negative, s_negative, s_negative, s_negative, s_negative]        1
[s_positive, s_positive, s_negative, s_positive]                    1
[s_positive, s_positive, s_negative, s_negative]                    1
[s_positive, s_negative, s_positive, s_positive]                    1
Name: Sentiment_Emotions, Length: 67, dtype: int64

blank are neutral.they remain intact

In [109]:
train_data['s_positive'] = np.zeros((len(train_data),1))
train_data['s_negative'] = np.zeros((len(train_data),1))
train_data['s_ambiguous'] = np.zeros((len(train_data),1))
train_data['s_neutral'] = np.zeros((len(train_data),1))

valid_data['s_positive'] = np.zeros((len(valid_data),1))
valid_data['s_negative'] = np.zeros((len(valid_data),1))
valid_data['s_ambiguous'] = np.zeros((len(valid_data),1))
valid_data['s_neutral'] = np.zeros((len(valid_data),1))


test_data['s_positive'] = np.zeros((len(test_data),1))
test_data['s_negative'] = np.zeros((len(test_data),1))
test_data['s_ambiguous'] = np.zeros((len(test_data),1))
test_data['s_neutral'] = np.zeros((len(test_data),1))



In [110]:
for i in ['s_positive', 's_negative', 's_ambiguous', 's_neutral']:
    train_data[i] = train_data['Sentiment_Emotions'].apply(lambda x: 1 if i in x else 0)
    valid_data[i] = valid_data['Sentiment_Emotions'].apply(lambda x: 1 if i in x else 0)
    test_data[i] = test_data['Sentiment_Emotions'].apply(lambda x: 1 if i in x else 0)

In [111]:
train_data.head()

,Text,Class,ID,Class List,Class Length,Emotions,GoEmotions,admiration,amusement,anger,...,e_fear,e_joy,e_sadness,e_surprise,e_neutral,Sentiment_Emotions,s_positive,s_negative,s_ambiguous,s_neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,...,0,0,0,0,1,[s_neutral],0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,...,0,0,0,0,1,[s_neutral],0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],0,0,1,...,0,0,0,0,0,[s_negative],0,1,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,0,...,1,0,0,0,0,[s_negative],0,1,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[annoyance],0,0,0,...,0,0,0,0,0,[s_negative],0,1,0,0


#Unnecessary Columns Removal

In [112]:
train_data.drop(["Class", "ID", "Class List", "Class Length", "Emotions"], axis=1, inplace=True)
valid_data.drop(["Class", "ID", "Class List", "Class Length", "Emotions"], axis=1, inplace=True)
test_data.drop(["Class", "ID", "Class List", "Class Length", "Emotions"], axis=1, inplace=True)

In [113]:
train_data.head()

,Text,GoEmotions,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,e_fear,e_joy,e_sadness,e_surprise,e_neutral,Sentiment_Emotions,s_positive,s_negative,s_ambiguous,s_neutral
0,My favourite food is anything I didn't have to...,[neutral],0,0,0,0,0,0,0,0,...,0,0,0,0,1,[s_neutral],0,0,0,1
1,"Now if he does off himself, everyone will thin...",[neutral],0,0,0,0,0,0,0,0,...,0,0,0,0,1,[s_neutral],0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,[anger],0,0,1,0,0,0,0,0,...,0,0,0,0,0,[s_negative],0,1,0,0
3,To make her feel threatened,[fear],0,0,0,0,0,0,0,0,...,1,0,0,0,0,[s_negative],0,1,0,0
4,Dirty Southern Wankers,[annoyance],0,0,0,1,0,0,0,0,...,0,0,0,0,0,[s_negative],0,1,0,0


#Data PreProcessing

In [114]:
def preprocess(sentence):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    sentence = re.sub('[^A-z]', ' ', sentence)
    negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                        'even though', 'yet']
    stop_words = [z for z in stop_words if z not in negative]
    preprocessed_tokens = [lemmatizer.lemmatize(contractions.fix(temp.lower())) for temp in sentence.split() if temp not in stop_words] #lemmatization
    return ' '.join([x for x in preprocessed_tokens]).strip()

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = preprocess(text)
    return text

In [115]:
train_data['Text'] = train_data['Text'].apply(lambda x: preprocess(x))

In [116]:
valid_data["Text"] = valid_data["Text"].apply(lambda x: preprocess(x))

In [117]:
test_data["Text"] = test_data["Text"].apply(text_preprocessing_pipeline)

In [118]:
pd.Series(train_data["Text"]).str.split().str.len().describe(percentiles=[0.05, 0.97])

count    43410.000000
mean         7.761253
std          3.956936
min          0.000000
5%           2.000000
50%          7.000000
97%         16.000000
max         33.000000
Name: Text, dtype: float64

In [119]:
train_data.head()

,Text,GoEmotions,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,e_fear,e_joy,e_sadness,e_surprise,e_neutral,Sentiment_Emotions,s_positive,s_negative,s_ambiguous,s_neutral
0,my favourite food anything i cook,[neutral],0,0,0,0,0,0,0,0,...,0,0,0,0,1,[s_neutral],0,0,0,1
1,now everyone think he laugh screwing people in...,[neutral],0,0,0,0,0,0,0,0,...,0,0,0,0,1,[s_neutral],0,0,0,1
2,why the fuck is bayless isoing,[anger],0,0,1,0,0,0,0,0,...,0,0,0,0,0,[s_negative],0,1,0,0
3,to make feel threatened,[fear],0,0,0,0,0,0,0,0,...,1,0,0,0,0,[s_negative],0,1,0,0
4,dirty southern wanker,[annoyance],0,0,0,1,0,0,0,0,...,0,0,0,0,0,[s_negative],0,1,0,0


In [120]:
ekman_list =['e_anger', 'e_disgust', 'e_fear', 'e_joy', 'e_sadness', 'e_surprise','e_neutral']
sentiment_list = ['s_positive', 's_negative', 's_ambiguous', 's_neutral']

In [121]:
train_text = list(train_data['Text'])
valid_text = list(valid_data['Text'])
test_text = list(test_data['Text'])

Model 1 : GoEmotions Taxonomy using RoBERTa Model

In [122]:
g_train_labels = train_data[emotion_list]
g_valid_labels = valid_data[emotion_list]
g_test_labels = test_data[emotion_list]

In [123]:
len(g_train_labels.columns)

28

#roberta-base model

In [126]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions")

sentence = ["The place looks awesome but i can't go there forever"]

model_outputs = classifier(sentence)

print(model_outputs)


[{'label': 'admiration', 'score': 0.8409616351127625}]


#EmoRoBERTa model

In [125]:

from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline

tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")

emotion = pipeline('sentiment-analysis',
                    model='arpanghoshal/EmoRoBERTa')

sentence = ["The place looks awesome but i can't go there forever"]
emotion_labels = emotion(sentence)
print(emotion_labels)


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


[{'label': 'admiration', 'score': 0.9872246980667114}]
